In [3]:
import os.path
from git import Repo

join = os.path.join

In [6]:
# rorepo is a Repo instance pointing to the git-python repository.
# For all you know, the first argument to Repo is a path to the repository
# you want to work with
path = "/home/hshahin/workspaces/OpenDSA"
repo = Repo(path)
assert not repo.bare

In [8]:
rw_dir = "/home/hshahin/workspaces/GitPython"
bare_repo = Repo.init(join(rw_dir, 'bare-repo'), bare=True)
assert bare_repo.bare

In [10]:
repo.config_reader()             # get a config reader for read-only access
cw = repo.config_writer()        # get a config writer to change configuration
cw.release()                     # call release() to be sure changes are written and locks are released

In [12]:
assert not bare_repo.is_dirty()  # check the dirty state
repo.untracked_files             # retrieve a list of untracked files
# ['my_untracked_file']

[u'config/CS3_LMSconf_local.json',
 u'config/CS3_LMSconf_vt_test.json',
 u'config/PL_LMSconf_local.json']

In [13]:
cloned_repo = repo.clone(join(rw_dir, '/home/hshahin/workspaces/GitPython_clone'))
assert cloned_repo.__class__ is Repo     # clone an existing repository
assert Repo.init(join(rw_dir, 'path/for/new/repo')).__class__ is Repo

In [15]:
assert os.path.isdir(cloned_repo.working_tree_dir)                    # directory with your work files
assert cloned_repo.git_dir.startswith(cloned_repo.working_tree_dir)   # directory containing the git repository
assert bare_repo.working_tree_dir is None                             # bare repositories have no working tree    

In [17]:
assert repo.head.ref == repo.heads.master                   # head is a symbolic reference pointing to master

In [18]:
# assert repo.tags['0.3.5'] == repo.tag('refs/tags/0.3.5')    # you can access tags in various ways too
assert repo.refs.master == repo.heads['master']             # .refs provides access to all refs, i.e. heads ...
assert repo.refs['origin/master'] == repo.remotes.origin.refs.master  # ... remotes ...
# assert repo.refs['0.3.5'] == repo.tags['0.3.5']             # ... and tags

In [22]:
new_branch = cloned_repo.create_head('feature2')               # create a new branch ...
assert cloned_repo.active_branch != new_branch                # which wasn't checked out yet ...
assert new_branch.commit == cloned_repo.active_branch.commit  # and which points to the checked-out commit

# It's easy to let a branch point to the previous commit, without affecting anything else
# Each reference provides access to the git object it points to, usually commits
assert new_branch.set_commit('HEAD~1').commit == cloned_repo.active_branch.commit.parents[0]

In [23]:
past = cloned_repo.create_tag('past', ref=new_branch,
                              message="This is a tag-object pointing to %s" % new_branch.name)
assert past.commit == new_branch.commit        # the tag points to the specified commit
assert past.tag.message.startswith("This is")  # and its object carries the message provided

now = cloned_repo.create_tag('now')            # This is a tag-reference. It may not carry meta-data
assert now.tag is None

In [24]:
assert now.commit.message != past.commit.message
# You can read objects directly through binary streams, no working tree required
assert (now.commit.tree / 'VERSION').data_stream.read().decode('ascii').startswith('1')

# You can traverse trees as well to handle all contained files of a particular commit
file_count = 0
tree_count = 0
tree = past.commit.tree
for item in tree.traverse():
    file_count += item.type == 'blob'
    tree_count += item.type == 'tree'
assert file_count and tree_count                        # we have accumulated all directories and files
assert len(tree.blobs) + len(tree.trees) == len(tree)   # a tree is iterable itself to traverse its children

AssertionError: 

In [27]:
import git
repo = git.Repo.clone_from(rw_dir, os.path.join(rw_dir, 'repo'), branch='master')

heads = repo.heads
master = heads.master       # lists can be accessed by name for convenience
master.commit               # the commit pointed to by head called master
master.rename('new_name')   # rename heads
master.rename('master')


ValueError: I/O operation on closed file

In [28]:
head = repo.head            # the head points to the active branch/ref
master = head.reference     # retrieve the reference the head points to
master.commit               # from here you use it as any other reference

<git.Commit "3f4d346b41d761247a53565b5b61ef546a622dc3">

In [29]:
log = master.log()
log[0]                      # first (i.e. oldest) reflog entry
log[-1]                     # last (i.e. most recent) reflog entry

dab7cad4a0dfd0129744f7a2744d3de722491b1a 3f4d346b41d761247a53565b5b61ef546a622dc3 Hosam Shahin <hosamlshahin@gmail.com> 1460395145 -0400	rebase finished: refs/heads/master onto 3f4d346b41d761247a53565b5b61ef546a622dc3


In [30]:
log[0]

0000000000000000000000000000000000000000 abad67adbbfc01d84ef5b780a3cc758f9ef52911 Hosam Shahin <hosamlshahin@gmail.com> 1454537040 -0500	branch: Created from refs/remotes/origin/master


In [31]:
log[-1]

dab7cad4a0dfd0129744f7a2744d3de722491b1a 3f4d346b41d761247a53565b5b61ef546a622dc3 Hosam Shahin <hosamlshahin@gmail.com> 1460395145 -0400	rebase finished: refs/heads/master onto 3f4d346b41d761247a53565b5b61ef546a622dc3


In [39]:
fifty_first_commits = list(repo.iter_commits('master'))
# assert len(fifty_first_commits) == 50
# this will return commits 21-30 from the commit list as traversed backwards master
ten_commits_past_twenty = list(repo.iter_commits('master', max_count=10, skip=20))
assert len(ten_commits_past_twenty) == 10
assert fifty_first_commits[20:30] == ten_commits_past_twenty

In [45]:
print len(fifty_first_commits)
# for i in fifty_first_commits:
#     print i.message
# # print fifty_first_commits

6422


In [4]:
from datetime import datetime
import time
import os.path
from git import Repo, Git

def get_epoch(year, month='01'):
    """
    calculate the epoch of first day of a year-month
    """
    pattern = '%Y.%m.%d %H:%M:%S'
    return int(time.mktime(time.strptime(str(year) + '.' + str(month) + '.01 00:00:00', pattern)))


def create_tags(path):
    '''
    takes repo path and creates tags for first commit in Jan and Jun. for every year
    # get the list of commits
    # get the latest commit date
    # current_year is the year from that date
    # loop through the list of commit to find the commit having a date equal or just after 1/1/current_year
    # once found create a tage with the current_year name on it AND
    # subtract 1 from the year and continue.

    '''

    repo = Repo(path)

    # get the list of commits
    commits = list(repo.iter_commits())

    # get the latest commit date, current_year is the year from that date
    current_year = datetime.fromtimestamp(commits[0].committed_date).year


    for idx, commit in enumerate(commits):
        current_year_01 = str(current_year)+'-01'
        current_year_06 = str(current_year)+'-06'

        if get_epoch(current_year, '01') > commit.committed_date and \
            int(time.time()) > get_epoch(current_year, '01')  and \
            idx !=0:
            if str(current_year_01) not in repo.tags and idx != 0:
                # print commits[idx-1].hexsha+' '+current_year_01
                past = repo.create_tag(current_year_01, ref=commits[idx-1],
                                  message="This is a tag to mark the first commit in year %s" % current_year_01)
            current_year = datetime.fromtimestamp(commit.committed_date).year

        if get_epoch(current_year, '06') > commit.committed_date and \
            int(time.time()) > get_epoch(current_year, '06') and \
            idx != 0:
            if str(current_year_06) not in repo.tags:
                # print commits[idx-1].hexsha+" "+current_year_06
                past = repo.create_tag(current_year_06, ref=commits[idx-1],
                                  message="This is a tag to mark the first commit in year %s" % current_year_06)
                

                
                
def checkout_tag(repo, git, tag_name):
    '''
    checks out a tag if it exists
    '''
    if tag_name in repo.tags:
        git.checkout(tag_name)
        
def delete_tags(path):

    repo = Repo(path)

    for tag in repo.tags:
        repo.delete_tag(tag)
                
            
path = '/home/hshahin/workspaces/OpenDSA'
create_tags(path)

In [60]:
# checkout tag
g = Git(path)
checkout_tag(repo, g, '2013-01')

In [64]:
# the current checked out commit
# repo.head.commit

AttributeError: 'Commit' object has no attribute 'tag'

In [5]:
delete_tags(path)

In [15]:
g.branch()

u'* (detached from 2013)\n  NewKA-2\n  master\n  master-2'

In [35]:
int(time.time())

1461266846

In [31]:
datetime.time(datetime.now())

datetime.time(15, 27, 18, 93190)